## 0. Imports

In [104]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import logging
from google.cloud.storage import Client
from sklearn.model_selection import train_test_split
import datetime
import gcsfs
import pickle
from preprocess import TextPreprocessor
from model_prediction import CustomModelPrediction
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

## 1. Config work

#### Update the following config ONLY

In [ ]:
MODEL_TIME_VERSION = '2022-06-03-15-42-52'

#### DO NOT update the following config :

In [5]:
!gcloud config set project {PROJECT}

Updated property [core/project].


In [ ]:
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT = PROJECT_ID[0]
BUCKET_NAME = f"{PROJECT}-machine-learning"
BUCKET= f"gs://{PROJECT}-machine-learning"
MODEL_DIR='level-1-models/valid_models/valid_model'+'_'+ MODEL_TIME_VERSION
PACKAGES_DIR='level-1-models/valid_packages'
REGION = 'europe-west9'
MODEL_NAME = 'tweet_sentiment_classifier'
temp_model = './model-'+ MODEL_TIME_VERSION +'/'
RUNTIME_VERSION='2.5' # tensorflow version
MODEL_REGION='europe-west1'

if not os.path.exists('./model-'+ MODEL_TIME_VERSION +'/'):
    os.makedirs('./model-'+ MODEL_TIME_VERSION +'/')
temp_model = './model-'+ MODEL_TIME_VERSION +'/'

In [45]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

## 4. Deployment

### 4.1. Prepare custom model prediction

In [76]:
%%writefile model_prediction.py
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.python.lib.io import file_io

class CustomModelPrediction(object):

    def __init__(self, model, processor):
        # Class gets instantiated with a trained model file and a persisted processor
        self._model = model
        self._processor = processor

    def _postprocess(self, predictions):
    # Create an output signature
        labels = ['negative', 'positive']
        return [
            {
            "label":labels[int(np.round(prediction))],
            "score":float(np.round(prediction,4))
            } for prediction in predictions]

    def predict(self, instances, **kwargs):
    # Clean the data, make predictions and postprocess
        preprocessed_data = self._processor.transform(instances)
        predictions =  self._model.predict(preprocessed_data)
        labels = self._postprocess(predictions)
        return labels

    @classmethod
    def from_path(cls, model_dir):
    # Load the keras model and the persisted processor
        
        print ('test model')
        model = tf.keras.models.load_model(model_dir,custom_objects={'tf': tf})
    
    # I know, pickle is bad and I should feel bad
    
        with file_io.FileIO(os.path.join(model_dir, 'processor_state.pkl'), 'rb') as f:
            processor = pickle.load(f)

        return cls(model, processor)

Overwriting model_prediction.py


#### Test

In [79]:
requests = (["God I hate the north","I love this"])

classifier = CustomModelPrediction.from_path(BUCKET+'/'+MODEL_DIR)
results = classifier.predict(requests)
results

### 4.2. Package model in tar.gz file

In [1]:
MODEL_TIME_VERSION

NameError: name 'MODEL_TIME_VERSION' is not defined

#### update VERSION in the cell below with the MODEL_TIME_VERSION above

In [88]:
%%writefile setup.py

from setuptools import setup

MODEL_NAME = "tweet_sentiment_classifier"
REQUIRED_PACKAGES = ['gcsfs']
VERSION = '2022-06-xx-xx-xx-xx'

setup(
    name=MODEL_NAME,
    packages=[],
    include_package_data=False,
    version=VERSION,
    scripts=["preprocess.py", "model_prediction.py"]
)

Overwriting setup.py


#### Wrap it up and copy to GCP

In [89]:
!python setup.py sdist --formats=gztar
!gsutil cp ./dist/{MODEL_NAME}-{MODEL_TIME_VERSION}.tar.gz {BUCKET}/{PACKAGES_DIR}/{MODEL_NAME}-{MODEL_TIME_VERSION}.tar.gz

running sdist
running egg_info
writing tweet_sentiment_classifier.egg-info/PKG-INFO
writing dependency_links to tweet_sentiment_classifier.egg-info/dependency_links.txt
writing top-level names to tweet_sentiment_classifier.egg-info/top_level.txt
reading manifest file 'tweet_sentiment_classifier.egg-info/SOURCES.txt'
writing manifest file 'tweet_sentiment_classifier.egg-info/SOURCES.txt'

running check


creating tweet_sentiment_classifier-2
creating tweet_sentiment_classifier-2/tweet_sentiment_classifier.egg-info
copying files to tweet_sentiment_classifier-2...
copying model_prediction.py -> tweet_sentiment_classifier-2
copying preprocess.py -> tweet_sentiment_classifier-2
copying setup.py -> tweet_sentiment_classifier-2
copying tweet_sentiment_classifier.egg-info/PKG-INFO -> tweet_sentiment_classifier-2/tweet_sentiment_classifier.egg-info
copying tweet_sentiment_classifier.egg-info/SOURCES.txt -> tweet_sentiment_classifier-2/tweet_sentiment_classifier.egg-info
copying tweet_sentiment_

## 5. Update model version on AI Platform

In [ ]:
VERSION_NAME='V_'+MODEL_TIME_VERSION.replace("-","_")

In [95]:
!gcloud ai-platform models list --region global

Using endpoint [https://ml.googleapis.com/]
NAME                        DEFAULT_VERSION_NAME
tweet_sentiment_classifier  V1


In [100]:
!gcloud beta ai-platform versions create {VERSION_NAME} \
--model {MODEL_NAME} \
--origin {BUCKET}/{MODEL_DIR} \
--python-version 3.7 \
--runtime-version {RUNTIME_VERSION} \
--package-uris {BUCKET}/{PACKAGES_DIR}/{MODEL_NAME}-{MODEL_TIME_VERSION}.tar.gz \
--prediction-class=model_prediction.CustomModelPrediction \
--region global 

Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


#### Set the new deployed version as default

In [122]:
!gcloud ai-platform versions set-default {VERSION_NAME} \
--model {MODEL_NAME}\
--region global

Using endpoint [https://ml.googleapis.com/]
createTime: '2022-06-03T20:54:11Z'
deploymentUri: gs://sound-splicer-351114-training-data/valid_model_2022-06-03-15-42-52
etag: opKbMl6YWSI=
isDefault: true
machineType: mls1-c1-m2
name: projects/sound-splicer-351114/models/tweet_sentiment_classifier/versions/V2
packageUris:
- gs://sound-splicer-351114-training-data/valid_package_2022-06-03-15-42-52/tweet_sentiment_classifier.tar.gz
predictionClass: model_prediction.CustomModelPrediction
pythonVersion: '3.7'
runtimeVersion: '2.5'
state: READY


## 6. Testing

In [118]:
requests = [
    "god this episode sucks",
    "meh, I kinda like it",
    "what were the writer thinking, omg it doesn't make any sense!",
    "omg! what a twist, who would'v though :o!",
    "woohoow, sansa for the win!"
]

# JSON format the requests
request_data = {'instances': requests}

# Authenticate and call CMLE prediction API 
credentials = GoogleCredentials.get_application_default()

In [116]:
%%time
api = discovery.build('ml', 'v1')
model_url = 'projects/{}/models/{}'.format(PROJECT, MODEL_NAME)
response = api.projects().predict(body=request_data, name=model_url).execute()
response["predictions"]

CPU times: user 11.8 ms, sys: 8.07 ms, total: 19.9 ms
Wall time: 78.2 ms


[{'label': 'negative', 'score': 0.050200000405311584},
 {'label': 'positive', 'score': 0.7918000221252441},
 {'label': 'negative', 'score': 0.373199999332428},
 {'label': 'negative', 'score': 0.19840000569820404},
 {'label': 'positive', 'score': 0.8440999984741211}]